In [35]:
# Load in data
import pandas as pd
data = pd.read_csv("Zillow_Airbnb.csv")
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# View data
data

,Unnamed: 0,RegionID,SizeRank,City,RegionType,StateName,Date,ZHVI,Listings
0,0,394355,29,"Austin, TX",msa,TX,2000-01-31,167608.803584,0
1,1,394355,29,"Austin, TX",msa,TX,2000-02-29,168157.892276,0
2,2,394355,29,"Austin, TX",msa,TX,2000-03-31,168656.684383,0
3,3,394355,29,"Austin, TX",msa,TX,2000-04-30,169450.508196,0
4,4,394355,29,"Austin, TX",msa,TX,2000-05-31,170048.593123,0
...,...,...,...,...,...,...,...,...,...
4721,4721,753899,2,"Los Angeles, CA",msa,CA,2022-10-31,887956.929337,14349
4722,4722,753899,2,"Los Angeles, CA",msa,CA,2022-11-30,884296.450518,13785
4723,4723,753899,2,"Los Angeles, CA",msa,CA,2022-12-31,880012.047421,13042
4724,4724,753899,2,"Los Angeles, CA",msa,CA,2023-01-31,871774.871645,11029


In [3]:
# Creat groupby object grouped by city
grouped = data.groupby(data['City'])

In [4]:
# Move data into dictionary with city as keys
groupdict = {}
for val in set(data['City'].values):
    groupdict[val] = grouped.get_group(val)

In [5]:
# Create list of first nonzero dates for listings; that is, date of first Airbnb listing
list = []
for key in set(groupdict.keys()):
    list.append(groupdict[key].iloc[groupdict[key]['Listings'].to_numpy().nonzero()[0]].iloc[0]['Date'])


In [6]:
# Convert date values to datetime type (currently strings)
for key in set(groupdict.keys()):
    groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')

/var/folders/60/bzl3rvnx2js1r_pvl2bzbtrw0000gn/T/ipykernel_53130/60717528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')
/var/folders/60/bzl3rvnx2js1r_pvl2bzbtrw0000gn/T/ipykernel_53130/60717528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')
/var/folders/60/bzl3rvnx2js1r_pvl2bzbtrw0000gn/T/ipykernel_53130/60717528.py:3: SettingWithCop

In [7]:
# Among all cities, the latest date of first listing is October 2011; as a result, for simplicity sake, we'll start with Dec 31, 2011, equivalent to the start of 2012.
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key][groupdict[key]['Date'] >= '2011-12-31']

In [8]:
# Remove columns not to be included in models
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key].drop(columns = ['Unnamed: 0', 'RegionID', 'SizeRank', 'City', 'RegionType', 'StateName'])

In [11]:
# Load in secondary features (federal funds rate, CPI, unemployment rate)
rate = pd.read_csv("FEDFUNDS.csv")
cpi = pd.read_csv("CPIAUCSL.csv")
unrate = pd.read_csv("UNRATE.csv")

In [12]:
# Convert date columns to datetime type
rate['DATE'] = pd.to_datetime(rate['DATE'], format='%m/%d/%y')
cpi['DATE'] = pd.to_datetime(cpi['DATE'], format='%m/%d/%y')
unrate['DATE'] = pd.to_datetime(unrate['DATE'], format='%m/%d/%y')

In [13]:
# Join new values into dictionary of dataframes
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key].merge(rate, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE', 'DATE'])
    groupdict[key] = groupdict[key].merge(cpi, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE'])
    groupdict[key] = groupdict[key].merge(unrate, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE'])

In [14]:
# Create dictionary to use for first difference model
diffdict = {}
for key in set(groupdict.keys()):
    diffdict[key] = groupdict[key].copy()

In [15]:
# Change values to difference between months for diffdict, remove first row (NaN value)
for key in set(diffdict.keys()):
    diffdict[key]['ZHVI'] = diffdict[key]['ZHVI'].diff()
    diffdict[key]['Listings'] = diffdict[key]['Listings'].diff()
    diffdict[key]['FEDFUNDS'] = diffdict[key]['FEDFUNDS'].diff()
    diffdict[key]['CPIAUCSL'] = diffdict[key]['CPIAUCSL'].diff()
    diffdict[key]['UNRATE'] = diffdict[key]['UNRATE'].diff()
    diffdict[key] = diffdict[key][diffdict[key]['Date'] > '2011-12-31']

In [16]:
# Define x's and y's for MLR model
reg_xs = {}
reg_ys = {}
for key in set(groupdict.keys()):
    reg_xs[key] = groupdict[key].drop(columns = ['Date', 'ZHVI'])
    reg_ys[key] = groupdict[key]['ZHVI']

In [17]:
# Define x's and y's for first difference model
diff_xs = {}
diff_ys = {}
for key in set(groupdict.keys()):
    diff_xs[key] = diffdict[key].drop(columns = ['Date', 'ZHVI'])
    diff_ys[key] = diffdict[key]['ZHVI']

In [81]:
# defining vif function
def vif(data):
    vif_df = pd.DataFrame()
    vif_df["predictors"] = data.columns
    vif_df["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]

    return(vif_df)

In [82]:
# Run MLR model; get p-values, R^2 values, and summary ourputs
import statsmodels.api as sm
reg_models = {}
p_values_reg = {}
R_sq_reg = {}
summaries_reg = {}
vif_reg = {}

for key in set(groupdict.keys()):
    
    reg_models[key] = sm.OLS(reg_ys[key],reg_xs[key])
    
    fii = reg_models[key].fit()
    
    p_values_reg[key] = fii.summary2().tables[1]['P>|t|']
    
    R_sq_reg[key] = fii.summary2().tables[0][1][6]
    
    summaries_reg[key] = fii.summary()
    
    vif_reg[key] = calc_vif(reg_xs[key])

In [83]:
# Checking for Multicolinearity using VIF in MLR model

# See if VIF for each IV is >= max(10,(1/(1-R_sq)))
MC_Issues_Reg = {}

for key in set(groupdict.keys()):
    vif_df = vif_reg[key]
    vif_df['vif_threshold'] = (1/(1-float(R_sq_reg[key])))
    vif_df['mc_issue'] = vif_df.apply(lambda row: 1 if row['VIF'] >= max(10,row['vif_threshold']) else 0, axis=1)
    if sum(vif_df['mc_issue']) > 0:
        MC_Issues_Reg[key] = 'Yes'
    else:
        MC_Issues_Reg[key] = 'No'
        
MC_Issues_Reg

{'Austin, TX': 'No',
 'Boston, MA': 'No',
 'Chicago, IL': 'No',
 'Dallas, TX': 'No',
 'New York, NY': 'No',
 'Portland, OR': 'No',
 'Salem, OR': 'No',
 'Los Angeles, CA': 'No',
 'San Diego, CA': 'No',
 'Minneapolis, MN': 'No',
 'Nashville, TN': 'No',
 'Denver, CO': 'No',
 'Columbus, OH': 'No',
 'Washington, DC': 'No',
 'Seattle, WA': 'No',
 'New Orleans, LA': 'No',
 'San Francisco, CA': 'No'}

In [89]:
# Run first difference model; get p-values, R^2 values, and summary ourputs
import statsmodels.api as sm
diff_models = {}
p_values_diff = {}
R_sq_diff = {}
summaries_diff = {}
vif_diff = {}
for key in set(groupdict.keys()):
    diff_models[key] = sm.OLS(diff_ys[key],diff_xs[key])
    fii = diff_models[key].fit()
    p_values_diff[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_diff[key] = fii.summary2().tables[0][1][6]
    summaries_diff[key] = fii.summary()
    vif_diff[key] = calc_vif(diff_xs[key])

In [90]:
# Checking for Multicolinearity using VIF in first difference model

# See if VIF for each IV is >= max(10,(1/(1-R_sq)))
MC_Issues_diff = {}

for key in set(groupdict.keys()):
    vif_df = vif_diff[key]
    vif_df['vif_threshold'] = (1/(1-float(R_sq_diff[key])))
    vif_df['mc_issue'] = vif_df.apply(lambda row: 1 if row['VIF'] >= max(10,row['vif_threshold']) else 0, axis=1)
    if sum(vif_df['mc_issue']) > 0:
        MC_Issues_diff[key] = 'Yes'
    else:
        MC_Issues_diff[key] = 'No'
        
MC_Issues_diff

{'Austin, TX': 'No',
 'Boston, MA': 'No',
 'Chicago, IL': 'No',
 'Dallas, TX': 'No',
 'New York, NY': 'No',
 'Portland, OR': 'No',
 'Salem, OR': 'No',
 'Los Angeles, CA': 'No',
 'San Diego, CA': 'No',
 'Minneapolis, MN': 'No',
 'Nashville, TN': 'No',
 'Denver, CO': 'No',
 'Columbus, OH': 'No',
 'Washington, DC': 'No',
 'Seattle, WA': 'No',
 'New Orleans, LA': 'No',
 'San Francisco, CA': 'No'}

In [20]:
# The coefficients aren't highly interpretable for determining variable impact beyond p-values; normalize variables to allow for interpretability
scaled_reg_xs = {}
scaled_reg_ys = {}
scaled_diff_xs = {}
scaled_diff_ys = {}
for key in set(groupdict.keys()):
    scaled_reg_xs[key] = (reg_xs[key]-reg_xs[key].mean())/reg_xs[key].std()
    scaled_reg_ys[key] = (reg_ys[key]-reg_ys[key].mean())/reg_ys[key].std()
    scaled_diff_xs[key] = (diff_xs[key]-diff_xs[key].mean())/diff_xs[key].std()
    scaled_diff_ys[key] = (diff_ys[key]-diff_ys[key].mean())/diff_ys[key].std()

In [94]:
# Run MLR model on scaled data; get p-values, R^2 values, and summary ourputs
reg_models_scaled = {}
p_values_reg_scaled = {}
R_sq_reg_scaled = {}
summaries_reg_scaled = {}
vif_reg_scaled = {}
for key in set(groupdict.keys()):
    reg_models_scaled[key] = sm.OLS(scaled_reg_ys[key],scaled_reg_xs[key])
    fii = reg_models_scaled[key].fit()
    p_values_reg_scaled[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_reg_scaled[key] = fii.summary2().tables[0][1][6]
    summaries_reg_scaled[key] = fii.summary()
    vif_reg_scaled[key] = calc_vif(scaled_reg_xs[key])

In [95]:
# Checking for Multicolinearity using VIF in MLR model

# See if VIF for each IV is >= max(10,(1/(1-R_sq)))
MC_Issues_Reg_Scaled = {}

for key in set(groupdict.keys()):
    vif_df = vif_reg_scaled[key]
    vif_df['vif_threshold'] = (1/(1-float(R_sq_reg_scaled[key])))
    vif_df['mc_issue'] = vif_df.apply(lambda row: 1 if row['VIF'] >= max(10,row['vif_threshold']) else 0, axis=1)
    if sum(vif_df['mc_issue']) > 0:
        MC_Issues_Reg_Scaled[key] = 'Yes'
    else:
        MC_Issues_Reg_Scaled[key] = 'No'
        
MC_Issues_Reg_Scaled

{'Austin, TX': 'No',
 'Boston, MA': 'No',
 'Chicago, IL': 'No',
 'Dallas, TX': 'No',
 'New York, NY': 'No',
 'Portland, OR': 'No',
 'Salem, OR': 'No',
 'Los Angeles, CA': 'No',
 'San Diego, CA': 'No',
 'Minneapolis, MN': 'No',
 'Nashville, TN': 'No',
 'Denver, CO': 'No',
 'Columbus, OH': 'No',
 'Washington, DC': 'No',
 'Seattle, WA': 'No',
 'New Orleans, LA': 'No',
 'San Francisco, CA': 'No'}

In [97]:
# Run first difference model on scaled data; get p-values, R^2 values, and summary ourputs
diff_models_scaled = {}
p_values_diff_scaled = {}
R_sq_diff_scaled = {}
summaries_diff_scaled = {}
vif_diff_scaled = {}
for key in set(groupdict.keys()):
    diff_models_scaled[key] = sm.OLS(scaled_diff_ys[key],scaled_diff_xs[key])
    fii = diff_models_scaled[key].fit()
    p_values_diff_scaled[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_diff_scaled[key] = fii.summary2().tables[0][1][6]
    summaries_diff_scaled[key] = fii.summary()
    vif_diff_scaled[key] = calc_vif(scaled_diff_xs[key])

In [98]:
# Checking for Multicolinearity using VIF in first difference model

# See if VIF for each IV is >= max(10,(1/(1-R_sq)))
MC_Issues_diff_scaled = {}

for key in set(groupdict.keys()):
    vif_df = vif_diff_scaled[key]
    vif_df['vif_threshold'] = (1/(1-float(R_sq_diff_scaled[key])))
    vif_df['mc_issue'] = vif_df.apply(lambda row: 1 if row['VIF'] >= max(10,row['vif_threshold']) else 0, axis=1)
    if sum(vif_df['mc_issue']) > 0:
        MC_Issues_diff_scaled[key] = 'Yes'
    else:
        MC_Issues_diff_scaled[key] = 'No'
        
MC_Issues_diff_scaled

{'Austin, TX': 'No',
 'Boston, MA': 'No',
 'Chicago, IL': 'No',
 'Dallas, TX': 'No',
 'New York, NY': 'No',
 'Portland, OR': 'No',
 'Salem, OR': 'No',
 'Los Angeles, CA': 'No',
 'San Diego, CA': 'No',
 'Minneapolis, MN': 'No',
 'Nashville, TN': 'No',
 'Denver, CO': 'No',
 'Columbus, OH': 'No',
 'Washington, DC': 'No',
 'Seattle, WA': 'No',
 'New Orleans, LA': 'No',
 'San Francisco, CA': 'No'}